<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguaje Natural

## Taller #4: Pre-Procesamiento de textos y Featuring Engineering 
`Fecha de entrega: Marzo 18, 2021. (Antes del inicio de la próxima clase).`

In [2]:
import re
import pandas as pd
from collections import Counter

from nltk.corpus import stopwords

In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
stopwords_sp = stopwords.words('spanish')

from sklearn.feature_extraction.text import CountVectorizer

# Punto 1: Pre-Procesamiento

- `[5 pts]` Leer el archivo `dialogos.csv` usando `pandas`
- `[10 pts]` Crear una nueva columna con el texto en minúscula, sin caracteres especiales ni números y sin palabras vacias
- `[7 pts]` Usando la nueva columna, ¿cuáles son las 10 palabras más usadas? 

In [28]:
Dialogos = pd.read_csv('dialogos (1).csv')
Dialogos.head

<bound method NDFrame.head of                                               Locución
0                                       Francamente no
1    Lo del canadiense. Por favor, como que vosotro...
2    Tienes razón. A lo mejor así te liberas de tu ...
3                                      Pues, tú sabrás
4                              Para mí que fue Krieger
..                                                 ...
954                        Yo qué sé, digo yo qué será
955  Lo han entendido mal. Las joyas que venden en ...
956                            Oooo... Me llamo Vivian
957                  ¿De dónde? ¿Del coño de tu madre?
958                       Ah, sí esto podría funcionar

[959 rows x 1 columns]>

In [46]:
Dialogos["pre-procesado"] = Dialogos["Locución"]

In [49]:
Dialogos["pre-procesado"] = Dialogos["pre-procesado"].str.lower()
Dialogos

,Locución,pre-procesado
0,Francamente no,francamente no
1,"Lo del canadiense. Por favor, como que vosotro...","lo del canadiense. por favor, como que vosotro..."
2,Tienes razón. A lo mejor así te liberas de tu ...,tienes razón. a lo mejor así te liberas de tu ...
3,"Pues, tú sabrás","pues, tú sabrás"
4,Para mí que fue Krieger,para mí que fue krieger
...,...,...
954,"Yo qué sé, digo yo qué será","yo qué sé, digo yo qué será"
955,Lo han entendido mal. Las joyas que venden en ...,lo han entendido mal. las joyas que venden en ...
956,Oooo... Me llamo Vivian,oooo... me llamo vivian
957,¿De dónde? ¿Del coño de tu madre?,¿de dónde? ¿del coño de tu madre?


In [52]:
for i in Dialogos.index: 
    Dialogos["pre-procesado"][i]= re.sub(r"[\W\d]", " ", Dialogos["pre-procesado"][i])

Dialogos

,Locución,pre-procesado
0,Francamente no,francamente no
1,"Lo del canadiense. Por favor, como que vosotro...",lo del canadiense por favor como que vosotro...
2,Tienes razón. A lo mejor así te liberas de tu ...,tienes razón a lo mejor así te liberas de tu ...
3,"Pues, tú sabrás",pues tú sabrás
4,Para mí que fue Krieger,para mí que fue krieger
...,...,...
954,"Yo qué sé, digo yo qué será",yo qué sé digo yo qué será
955,Lo han entendido mal. Las joyas que venden en ...,lo han entendido mal las joyas que venden en ...
956,Oooo... Me llamo Vivian,oooo me llamo vivian
957,¿De dónde? ¿Del coño de tu madre?,de dónde del coño de tu madre


In [54]:
for j in Dialogos.index: 
        Dialogos["pre-procesado"][j]=' '.join([i for i in Dialogos["pre-procesado"][j].split() if i not in stopwords_sp])

Dialogos

,Locución,pre-procesado
0,Francamente no,francamente
1,"Lo del canadiense. Por favor, como que vosotro...",canadiense favor tiraríais allí si pudieseis
2,Tienes razón. A lo mejor así te liberas de tu ...,razón mejor así liberas energía nagativa
3,"Pues, tú sabrás",pues sabrás
4,Para mí que fue Krieger,krieger
...,...,...
954,"Yo qué sé, digo yo qué será",sé digo
955,Lo han entendido mal. Las joyas que venden en ...,entendido mal joyas venden canales siquiera in...
956,Oooo... Me llamo Vivian,oooo llamo vivian
957,¿De dónde? ¿Del coño de tu madre?,dónde coño madre


In [56]:
Palabras_Dialogos = pd.DataFrame(Dialogos["pre-procesado"].str.split(' ', expand=True).stack(), columns=["pre-procesado"])
Palabras_Dialogos=Palabras_Dialogos.groupby(["pre-procesado"])["pre-procesado"].count()
Palabras_Dialogos=pd.DataFrame(Palabras_Dialogos)
Palabras_Dialogos.index.name = None
Palabras_Dialogos.sort_values(by='pre-procesado', ascending=False).head(10)

,pre-procesado
,48
bien,36
si,34
cómo,34
así,30
oh,28
vamos,26
sarcasmobol,24
aquí,24
bueno,24


# Punto 2: Representación vectorial

- `[15 pts]` Crear una bolsa de palabras (BoW) del corpus usando la columna pre-procesada (Usando la función de `sklearn`)
- `[10 pts]` ¿Cuántas palabras hay en el vocabulario? (Usando la función de `sklearn`)

In [57]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
Dialogos["bow"] = ""
bow_rep=count_vect.fit_transform(Dialogos["pre-procesado"])
Dialogos["bow"]=[row for row in bow_rep.toarray()]
Dialogos

,Locución,pre-procesado,bow
0,Francamente no,francamente,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"Lo del canadiense. Por favor, como que vosotro...",canadiense favor tiraríais allí si pudieseis,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,Tienes razón. A lo mejor así te liberas de tu ...,razón mejor así liberas energía nagativa,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"Pues, tú sabrás",pues sabrás,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,Para mí que fue Krieger,krieger,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...
954,"Yo qué sé, digo yo qué será",sé digo,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
955,Lo han entendido mal. Las joyas que venden en ...,entendido mal joyas venden canales siquiera in...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
956,Oooo... Me llamo Vivian,oooo llamo vivian,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
957,¿De dónde? ¿Del coño de tu madre?,dónde coño madre,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [59]:
Cantidad_Palabras=count_vect.vocabulary_
len(Cantidad_Palabras)

2115

2115

# Punto 3: 🤔

- `[3pts]` ¿Cuándo es una mejor idea usar una bolsa de n-gramas en vez de una bolsa de palabras?

#### Es mejor idea usar la bolsa de n-gramas en vez de una bolsa de palabras,  cuando se quiera enfocar el orden y la informacion del contexto


# BONUS Punto 4: 😜

- `[2pt]` ¿A qué pertenecen los dialogos de ese archivo? 

####  South Park y Archer